In [0]:
! pip install pm4py

     |████████████████████████████████| 389kB 1.3MB/s 
     |████████████████████████████████| 27.7MB 146kB/s 
     |████████████████████████████████| 39.2MB 104kB/s 
  Created wheel for ciso8601: filename=ciso8601-2.1.2-cp36-cp36m-linux_x86_64.whl size=27676 sha256=d815a932c992a9a72e9779d537b9dea1cad939780a3bc0acaba5dda980ad54d1
  Stored in directory: /root/.cache/pip/wheels/3b/13/4a/e6b972b23b1d7fca074f2ef681b3c819123b842cf1b4de8627
Successfully built ciso8601


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.util import sorting
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.objects.log.util import get_log_representation
import csv



#open empty file to write
train_file = open("/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/train_log.txt","w")
test_file = open("/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/test_log.txt","w")
#import event log
log = xes_import_factory.apply('/content/drive/My Drive/didactoriko/datasets/BPI Challenge 2017.xes')
#print(log)

activities = attributes_filter.get_attribute_values(log, "concept:name")
print(activities)
#act = 0
#for key in activities:
#  print(act)
#  act = act + activities[key]
#print(act)
#sort event log based on timestamp
log = sorting.sort_timestamp(log)


#str_trace_attributes = []
#str_event_attributes = ["concept:name"]
#num_trace_attributes = []
#num_event_attributes = ["amount"]
#data, feature_names = get_log_representation.get_representation(
#                           log, str_trace_attributes, str_event_attributes,
#                           num_trace_attributes, num_event_attributes)

#print(data)
i=0
print(len(log))
for trace in log:
  for event in trace:
    if event['lifecycle:transition'] == 'start':
      #print(event['concept:name'], event['time:timestamp'])
      data = event['concept:name'].replace(' ','_')
      if i <= 0.8 * len(log):
        train_file.write(data)
        train_file.write(' ')
      else:
        test_file.write(data)
        test_file.write(' ') 
  if i <= 0.8 * len(log):
    train_file.write('\n')  
  else:
    test_file.write('\n') 
  i+=1

train_file.close()
test_file.close()

{'A_Create Application': 31509, 'A_Submitted': 20423, 'W_Handle leads': 47264, 'W_Complete application': 148900, 'A_Concept': 31509, 'A_Accepted': 31509, 'O_Create Offer': 42995, 'O_Created': 42995, 'O_Sent (mail and online)': 39707, 'W_Call after offers': 191092, 'A_Complete': 31362, 'W_Validate application': 209496, 'A_Validating': 38816, 'O_Returned': 23305, 'W_Call incomplete files': 168529, 'A_Incomplete': 23055, 'O_Accepted': 17228, 'A_Pending': 17228, 'A_Denied': 3753, 'O_Refused': 4695, 'O_Cancelled': 20898, 'A_Cancelled': 10431, 'O_Sent (online only)': 2026, 'W_Assess potential fraud': 3282, 'W_Personal Loan collection': 22, 'W_Shortened completion ': 238}
31509


In [0]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,TimeDistributed, Activation
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import LSTM, Input, Bidirectional
from keras.models import load_model
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy



LSTM_CELLS = 50
steps = 2


#/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/my_log_acron.txt
with open('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/train_log.txt', 'r') as myfile:
    data=myfile.read()

#Just for Eating/Drinking
#data = data.replace('Eating/Drinking','EatDrink')

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
#reads the words in data and gives an index for every words based on frequency
tokenizer.fit_on_texts([data])
print('Word index: ')
print(tokenizer.word_index)

#replace every word in the text to correspoding word index - returns list of list with one element so use [0] to get the one and only first list
encoded = tokenizer.texts_to_sequences([data])[0]
print('encoded: \n')
print(encoded)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# LSTM 3 timesteps - prepare data - encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
print('Sequences before padding: \n')
print(sequences)

max_length = max([len(seq) for seq in sequences])    #max_length is 3
# Pad sequence to be of the same length
# length of sequence must be 3 (maximum)
# 'pre' or 'post': pad either before or after each sequence
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)   													
print('Sequences after padding: \n')
print(sequences)
#convert list to array to get X,y train
sequences = array(sequences)
print('Sequences array: \n')
print(sequences)
X, y = sequences[:,:-1],sequences[:,-1]


#convert y to binary vectors
y = to_categorical(y, num_classes=vocab_size)



model = Sequential()
#the first layer
# - the largest integer (i.e. word index) in the input should be no larger than vocabulary size 
# - The Embedding layer is initialized with random weights and will learn an embedding for all of the words in the training dataset.
# - output_dim (50): This is the size of the vector space in which words will be embedded (size of the embedding vectors). It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.
# - input_length: This is the length of input sequences (here is 2)
# The Embedding layer has weights that are learned. If you save your model to file, this will include weights for the Embedding layer.
# The output of the Embedding layer is a 2D vector with one embedding for each word in the input sequence of words (input document).
# If you wish to connect a Dense layer directly to an Embedding layer, you must first flatten the 2D output matrix to a 1D vector using the Flatten layer.

model.add(Embedding(vocab_size+1, LSTM_CELLS, input_length=max_length-1))

model.add(LSTM(vocab_size))
model.add(Dropout(0.1))
model.add(Dense(vocab_size, activation='softmax'))
                                                                                                             
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2,batch_size = 50)

# Evaluate network
print('LSTM Network Evaluation:\n')
print(X)
print(y)
loss = model.evaluate(X, y, verbose=0)

print(loss)

print(model.summary())
model.save('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/lstm_model.h5')  # creates a HDF5 file 
del model  # deletes the existing model
#print('X: ')
#print(X)
#print('shape: ')
#print(X.shape[0],X.shape[1])
#this is #ofsamples, 2 -timesteps, 1-output feautre






Word index: 
{'w_validate_application': 1, 'w_call_after_offers': 2, 'w_complete_application': 3, 'w_call_incomplete_files': 4, 'w_handle_leads': 5, 'w_assess_potential_fraud': 6, 'w_shortened_completion_': 7, 'w_personal_loan_collection': 8}
encoded: 

[3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 1, 5, 3, 2, 1, 4, 1, 4, 1, 3, 2, 3, 2, 3, 2, 1, 4, 3, 2, 1, 3, 2, 1, 4, 3, 2, 3, 2, 1, 4, 1, 3, 2, 2, 1, 4, 1, 3, 2, 3, 2, 1, 4, 1, 4, 1, 3, 2, 5, 3, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 3, 2, 1, 4, 3, 2, 1, 4, 5, 3, 2, 1, 3, 2, 1, 4, 3, 2, 1, 3, 2, 3, 2, 3, 2, 5, 3, 2, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 1, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 4, 1, 3, 2, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 4, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 4, 3, 2, 1, 4, 1, 4, 3, 2, 1, 4, 1, 4, 1, 5, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 3, 2, 3, 2, 5, 3, 2, 1, 4

In [0]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,TimeDistributed, Activation
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import LSTM, Input, Bidirectional
from keras.models import load_model
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy



LSTM_CELLS = 50


#/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/my_log_acron.txt
with open('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/train_log.txt', 'r') as myfile:
    data=myfile.read()

#Just for Eating/Drinking
#data = data.replace('Eating/Drinking','EatDrink')

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
#reads the words in data and gives an index for every words based on frequency
tokenizer.fit_on_texts([data])
print('Word index: ')
print(tokenizer.word_index)

#replace every word in the text to correspoding word index - returns list of list with one element so use [0] to get the one and only first list
encoded = tokenizer.texts_to_sequences([data])[0]
print('encoded: \n')
print(encoded)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# LSTM 3 timesteps - prepare data - encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
print('Sequences before padding: \n')
print(sequences)

max_length = max([len(seq) for seq in sequences])    #max_length is 3
# Pad sequence to be of the same length
# length of sequence must be 3 (maximum)
# 'pre' or 'post': pad either before or after each sequence
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)   													
print('Sequences after padding: \n')
print(sequences)
#convert list to array to get X,y train
sequences = array(sequences)
print('Sequences array: \n')
print(sequences)
X, y = sequences[:,:-1],sequences[:,-1]


#convert y to binary vectors
y = to_categorical(y, num_classes=vocab_size)
X = X.reshape((X.shape[0], X.shape[1], 1))
#print(X.shape)
#print(X)
#y = y.reshape(1,3,1)

model = Sequential()
model.add(Bidirectional(LSTM(vocab_size), input_shape=(2, 1)))
model.add(Dropout(0.1))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=[categorical_accuracy])
model.fit(X, y, epochs=500, verbose=2,batch_size = 10)

# Evaluate network
print('Evaluate training')
print(X)
print(y)
loss = model.evaluate(X, y, verbose=0)
print('Bidirectional Network Evaluation:\n')
print(loss)

model.summary()
model.save('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/bidirectional_model.h5')  # creates a HDF5 file 
#Save model to drive
#torch.save(model.state_dict(), path)

del model  # deletes the existing model


Word index: 
{'w_validate_application': 1, 'w_call_after_offers': 2, 'w_complete_application': 3, 'w_call_incomplete_files': 4, 'w_handle_leads': 5, 'w_assess_potential_fraud': 6, 'w_shortened_completion_': 7, 'w_personal_loan_collection': 8}
encoded: 

[3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 1, 5, 3, 2, 1, 4, 1, 4, 1, 3, 2, 3, 2, 3, 2, 1, 4, 3, 2, 1, 3, 2, 1, 4, 3, 2, 3, 2, 1, 4, 1, 3, 2, 2, 1, 4, 1, 3, 2, 3, 2, 1, 4, 1, 4, 1, 3, 2, 5, 3, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 3, 2, 1, 4, 3, 2, 1, 4, 5, 3, 2, 1, 3, 2, 1, 4, 3, 2, 1, 3, 2, 3, 2, 3, 2, 5, 3, 2, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 1, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 4, 1, 3, 2, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 1, 3, 2, 1, 4, 1, 3, 2, 1, 4, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 3, 2, 1, 4, 1, 4, 1, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 4, 3, 2, 1, 4, 1, 4, 3, 2, 1, 4, 1, 4, 1, 5, 3, 2, 3, 2, 1, 4, 1, 3, 2, 1, 3, 2, 1, 4, 3, 2, 3, 2, 5, 3, 2, 1, 4

KeyboardInterrupt: ignored

In [0]:
import math
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed, Activation
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import load_model


#predict sequence of n_words activities
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
  #get input activity
  in_text = seed_text
  #print('in_text',in_text,'\n')
  #for the number of activities on sequence you want to predict
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    #pad if less than max text length
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    #print('in text ',in_text)
    #predict one activity
    yhat = model.predict_classes(encoded, verbose=0)
    out_word = ''
    for word, index in tokenizer.word_index.items():
      #convert predicted activity to word
      if index == yhat:
        #print('Word',word,'\n')
        out_word = word
        break
    #feed the next input with the sequence of activities
    in_text += ' ' + out_word
    
  return in_text
 

with open('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/test_log.txt', 'r') as myfile:
    data=myfile.read()
#data = data.replace('Eating/Drinking','EatDrink')
#print(data)

#/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/my_log_acron.txt
with open('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/train_log.txt', 'r') as myfile:
    traindata=myfile.read()

#Just for Eating/Drinking
#data = data.replace('Eating/Drinking','EatDrink')
#Be carefull!!! Tokenize with training data!!!
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
#reads the words in data and gives an index for every words based on frequency
tokenizer.fit_on_texts([traindata])
print('Word index: ')
print(tokenizer.word_index)


encoded = tokenizer.texts_to_sequences([data])[0]

vocab_size = len(tokenizer.word_index) + 1
sequences = list()

for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]

#convert y to binary vectors
yl = to_categorical(y, num_classes=vocab_size)


#load model from drive
#model.load_state_dict(torch.load(path))

#load trained model
model = load_model('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/lstm_model.h5')



# Evaluate network
print('LSTM Network Evaluation:\n')
print(X)
print(y)
loss = model.evaluate(X, yl, verbose=0)
print(loss)


print('LSTM Results: ')
print ('\n')
#predict for every activity the next three activities in sequence 
#print(generate_seq(model, tokenizer, max_length-1, 'deskwork' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'housework' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'socializing' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'eatingdrinking' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'personalgrooming' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'transportation' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'mealpreparation' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'movement' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'relaxing' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'shopping' , 3))
#print(generate_seq(model, tokenizer, max_length-1, 'sport' , 3))

#Uncomment for sequence prediction
for i in tokenizer.word_index:
  #print(tokenizer.index_word)
  print(generate_seq(model, tokenizer, max_length-1, i , 3))



#print(X) 
#this is #ofsamples, 2 -timesteps, 1-output feautre
#print('X:',X)
#print('reshaped X:',X)
print('first X:',X[0])
print('res X:',X[0])

#print(X)
#load trained model
model = load_model('/content/drive/My Drive/didactoriko/python/LSTM_BIDIRECTIONAL_RNN/bidirectional_model.h5')


print('Bidirectional Results: ')
print ('\n') 
#for i in tokenizer.word_index:
#	print(generate_seq(model, tokenizer, max_length-1, i , 1))
cnt = 0
for i in range(len(X)):
  yhat = model.predict_classes(X[i].reshape(1,2,1))
  #print('Expected:', y[i], 'Predicted', yhat)
  if (y[i] == yhat):
    cnt += 1

print('Total successful: ',cnt,' out of ', len(X), 'Percentage: ', cnt/len(X))

# Evaluate network
X = X.reshape((X.shape[0], X.shape[1], 1))
loss = model.evaluate(X, yl, verbose=0)
print('Bidirectional Network Evaluation:\n')
print(loss)



Word index: 
{'w_validate_application': 1, 'w_call_after_offers': 2, 'w_complete_application': 3, 'w_call_incomplete_files': 4, 'w_handle_leads': 5, 'w_assess_potential_fraud': 6, 'w_shortened_completion_': 7, 'w_personal_loan_collection': 8}
LSTM Network Evaluation:

[[3 2]
 [2 1]
 [1 4]
 ...
 [1 4]
 [4 1]
 [1 3]]
[1 4 1 ... 1 3 2]
[0.6612843423263488, 0.7257203039372917]
LSTM Results: 


w_validate_application w_call_incomplete_files w_validate_application w_complete_application
w_call_after_offers w_validate_application w_call_incomplete_files w_validate_application
w_complete_application w_call_after_offers w_validate_application w_call_incomplete_files
w_call_incomplete_files w_validate_application w_complete_application w_call_after_offers
w_handle_leads w_complete_application w_call_after_offers w_validate_application
w_assess_potential_fraud w_validate_application w_call_incomplete_files w_validate_application
w_shortened_completion_ w_call_after_offers w_validate_application w

OSError: ignored